This kernel uses Tensorflow to define, train, and evaluate a model.

This is part of a larger solution found at: https://www.kaggle.com/ohbewise/a-rsna-mri-solution-from-dicom-to-submission

In [ ]:
import pickle
import tensorflow as tf
import matplotlib.pyplot as plt

scan_types    = ['FLAIR','T1w','T1wCE','T2w']

In [ ]:
# Define, train, and evaluate model
# source: https://keras.io/examples/vision/3D_image_classification/
def get_model(width=128, height=128, depth=64, name='3dcnn'):
    """Build a 3D convolutional neural network model."""

    inputs = tf.keras.Input((width, height, depth, 1))

    x = tf.keras.layers.Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = tf.keras.layers.MaxPool3D(pool_size=2)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = tf.keras.layers.MaxPool3D(pool_size=2)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = tf.keras.layers.MaxPool3D(pool_size=2)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = tf.keras.layers.MaxPool3D(pool_size=2)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.GlobalAveragePooling3D()(x)
    x = tf.keras.layers.Dense(units=512, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.3)(x)

    outputs = tf.keras.layers.Dense(units=1, activation="sigmoid")(x)

    # Define the model.
    model = tf.keras.Model(inputs, outputs, name=name)
    
    # Compile model.
    initial_learning_rate = 0.0001
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
    )
    model.compile(
        loss="binary_crossentropy",
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        metrics=["acc"],
    )
    
    return model

In [ ]:
for scan_type in scan_types:
    # load train_dataset dataset
    tf_data_path = f'../input/nifti-to-split-dataset-with-nibabel/datasets/{scan_type}_train_dataset'
    with open(tf_data_path + '/element_spec', 'rb') as in_:
        es = pickle.load(in_)
    train_dataset = tf.data.experimental.load(tf_data_path, es, compression='GZIP')
    
    # load validation_dataset
    tf_data_path = f'../input/nifti-to-split-dataset-with-nibabel/datasets/{scan_type}_validation_dataset'
    with open(tf_data_path + '/element_spec', 'rb') as in_:
        es = pickle.load(in_)
    validation_dataset = tf.data.experimental.load(tf_data_path, es, compression='GZIP')

    # Get Model
    model = get_model(width=128, height=128, depth=64,name=scan_type)
    
    # Define callbacks.
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
        f'{scan_type}_3d_image_classification.h5', save_best_only=True
    )
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor="val_acc", patience=15)

    epochs = 100
    model.fit(
        train_dataset,
        validation_data=validation_dataset,
        epochs=epochs,
        shuffle=True,
        verbose=2,
        callbacks=[checkpoint_cb, early_stopping_cb],
    )
    
    #save model
    model.save(f'./models/{scan_type}')
    
    # show metrics
    fig, ax = plt.subplots(1, 2, figsize=(20, 3))
    ax = ax.ravel()

    for i, metric in enumerate(["acc", "loss"]):
        ax[i].plot(model.history.history[metric])
        ax[i].plot(model.history.history["val_" + metric])
        ax[i].set_title("{} Model {}".format(scan_type, metric))
        ax[i].set_xlabel("epochs")
        ax[i].set_ylabel(metric)
        ax[i].legend(["train", "val"])